In [1]:
%env CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
%env USER=neo4j
%env PW=teampandas

env: CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
env: USER=neo4j
env: PW=teampandas


In [23]:
CONNECT="$CYPHER -u $USER -p $PW"

In [24]:
!{CONNECT} "SHOW DATABASES"

+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

2 rows available after 2 ms, consumed after another 4 ms


In [25]:
!{CONNECT} "MATCH (n) DETACH DELETE n"

0 rows available after 15 ms, consumed after another 0 ms


In [26]:
!cat /home/jupyter/airbnb/load_data.cypher | {CONNECT} --format plain

COUNT(l)
5835
COUNT(a)
42
COUNT(n)
41
COUNT(h)
4633
COUNT(u)
55917
COUNT(r)
62976


In [27]:
!{CONNECT} "MATCH (n) RETURN count(n)"

+----------+
| count(n) |
+----------+
| 129444   |
+----------+

1 row available after 8 ms, consumed after another 1 ms


In [28]:
!{CONNECT} "MATCH (n) RETURN distinct labels(n), count(n)"

+-----------------------------+
| labels(n)        | count(n) |
+-----------------------------+
| ["Review"]       | 62976    |
| ["User"]         | 55917    |
| ["Listing"]      | 5835     |
| ["Amenity"]      | 42       |
| ["Neighborhood"] | 41       |
| ["Host"]         | 4633     |
+-----------------------------+

6 rows available after 13 ms, consumed after another 79 ms


### Q1. How many hosts are located in "Austin, Texas, United States"?

In [29]:
!{CONNECT} "MATCH (h:Host) where h.location= 'Austin, Texas, United States' return count(h)"

+----------+
| count(h) |
+----------+
| 3774     |
+----------+

1 row available after 22 ms, consumed after another 29 ms


### Q2. Which listings does host_id = "4641823" have? Return the listing name, property_type, price, and availability_365 sorted by price. Limit the results to 10.

In [30]:
!{CONNECT} "MATCH (h:Host)-[:HOSTS]-(l:Listing) where h.host_id='4641823' return l.name,l.property_type,l.price, l.availability_365 order by l.price limit 10"

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "1BR Convenient Austin Condo "        | "Apartment"     | 93.0    | 354                |
| "1BR Inviting Downtown Condo, Pool"   | "Apartment"     | 99.0    | 335                |
| "2BR/1.5BA Charming House Off SoCo"   | "House"         | 110.0   | 357                |
| "2BR Prime East-Side Downtown"        | "House"         | 121.0   | 341                |
| "1BR SoCo Treehouse Studio"           | "House"         | 129.0   | 327                |
| "1BR/1.5BA East 6th, Colorful 2Story" | "Apartment"     | 134.0   | 344                |
| "3BR Prestigious Home Travis Heights" | "House"         | 138.0   | 0                  |
| "1BR/1.5BA Perfectly Located Casita"  | "House"         | 140.0   | 351                |

### Q3. Which users wrote a review for listing_id = "5293632"? Return the user’s id and name sorted alphabetically by name. Limit the results to 10.

In [31]:
!{CONNECT} "MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(l:Listing) where l.listing_id='5293632' return u.user_id, u.name order by u.name limit 10"

+--------------------------------+
| u.user_id  | u.name            |
+--------------------------------+
| "18286390" | "Annie"           |
| "30193020" | "Carole"          |
| "16497582" | "Cory"            |
| "35022795" | "Craig And Trina" |
| "13281665" | "Dianne"          |
| "29601600" | "Hannah"          |
| "11940539" | "Jacob"           |
| "3213433"  | "Jessie"          |
| "41722221" | "Johannes"        |
| "28480571" | "Ju-Ju"           |
+--------------------------------+

10 rows available after 107 ms, consumed after another 5 ms


### Q4. Which users wrote a review for any listing which has the amenities "Washer" and "Dryer"? Return the user’s id and name sorted alphabetically by name. Limit the results to 10.

In [91]:
!{CONNECT} "MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]-> (l:Listing)-[:HAS]->(a:Amenity) where a.name='Washer' MATCH (l:Listing)-[:HAS]->(b:Amenity) where b.name='Washer'  return u.user_id, u.name order by u.name limit 10"


+-------------------------------------+
| u.user_id  | u.name                 |
+-------------------------------------+
| "6524431"  | "'Ley"                 |
| "8026901"  | "(We Are) Bonnie & Ky" |
| "7802424"  | "(email hidden)"       |
| "12694638" | "(email hidden)"       |
| "12694638" | "(email hidden)"       |
| "5164376"  | "(email hidden)"       |
| "13381969" | "(email hidden)"       |
| "7802424"  | "(email hidden)"       |
| "11656027" | "(email hidden)"       |
| "15315643" | "(email hidden)"       |
+-------------------------------------+

10 rows available after 83 ms, consumed after another 241 ms


In [88]:
!{CONNECT} "MATCH (a: Amenity{{ name: 'Washer'}}) <- [:HAS] -  (l: Listing) - [:HAS] -> (b: Amenity{{name: 'Dryer'}})<-[:HAS]-(r:Review) return a,b "

+-------+
| a | b |
+-------+
+-------+

0 rows available after 73 ms, consumed after another 0 ms


### Q5. Which listings have 3 bedrooms and are located in the Clarksville neighborhood? Return the listing name, property_type, price, and availability_365 sorted by price. Limit the results to 5.


In [44]:
!{CONNECT} "MATCH (l:Listing)-[:IN_NEIGHBORHOOD]->(n:Neighborhood) where n.name ='Clarksville' return l.name, l.property_type, l.price, l.availability_365 order by l.price limit 5"

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "private bedroom in shared apartment" | "Apartment"     | 30.0    | 41                 |
| "Cozy Retreat Near Downtown"          | "Apartment"     | 39.0    | 363                |
| "Share Our Apartment in West Austin!" | "Apartment"     | 45.0    | 98                 |
| "Cozy Room in Perfect Area!"          | "House"         | 45.0    | 340                |
| "Retro Central Austin Suite"          | "House"         | 48.0    | 53                 |
+----------------------------------------------------------------------------------------+

5 rows available after 49 ms, consumed after another 3 ms


### Q6. Which amenities are the most common? Return the name of the amenity and its frequency. Sort the results by count in descending order. Limit the results to 5.

In [45]:
!{CONNECT} "MATCH (a:Amenity) RETURN a.name, count(*)  ORDER BY count(*) DESC LIMIT 5"

+--------------------------------+
| a.name              | count(*) |
+--------------------------------+
| "Internet"          | 1        |
| "Wireless Internet" | 1        |
| "Air Conditioning"  | 1        |
| "Kitchen"           | 1        |
| "Cable TV"          | 1        |
+--------------------------------+

5 rows available after 31 ms, consumed after another 2 ms


### Q7. Which neighborhoods have the highest number of listings? Return the neighborhood’s name and zip code (neighborhood_id) along with the number of listings they have sorted by the number of listings in descending order. Limit the results to 5.

In [51]:
!{CONNECT} "MATCH (l:Listing)-[:IN_NEIGHBORHOOD]->(n:Neighborhood) RETURN n.name, n.neighborhood_id, count(l) ORDER BY count(l) DESC LIMIT 5"

+-------------------------------------------------+
| n.name           | n.neighborhood_id | count(l) |
+-------------------------------------------------+
| NULL             | "78704"           | 1601     |
| NULL             | "78702"           | 797      |
| "Clarksville"    | "78703"           | 419      |
| "East Riverside" | "78741"           | 414      |
| NULL             | "78745"           | 328      |
+-------------------------------------------------+

5 rows available after 38 ms, consumed after another 36 ms
